# Exercise: Build A Collaborative Filtering Movie Recommender System with Surprise

In this exercise, you will build a collaborative filtering movie recommender system using either the `KNNWithMeans` or `SVD` algorithm from the `Scikit-Surprise` library. The dataset used is the combined Movielens dataset contained in the  `ratings_movies.csv` file that you have obtained at the end of the [previous exercise](./02_exercise_most_popular.ipynb#save-the-combined-dataframe-to-a-csv-file) after merging the `ratings.csv` and `movies.csv` files from the Movielens dataset. 

**Instructions**:
1. Load the combined Movielens dataset from the `ratings_movies.csv` file.
2. Create a `Reader` object, mapping the rating scale from 0.5 to 5.
3. Load the dataset into a `Dataset` object, using the columns `['userId', 'movieId', 'rating']`.
4. Split the dataset into training and testing sets using the `train_test_split` method from the `model_selection` module.
5. Train a collaborative filtering model using either the `KNNWithMeans` or `SVD` algorithm.
6. Make predictions on the test set and evaluate the model using the `RMSE` metric.
7. Generate top-N movie recommendations for a given user ID using the `get_top_n` function provided in
   + [KNN notebook](./03_collaborative_filtering_similarity.ipynb#k-nearest-neighbors).
   + [SVD notebook](./03_collaborative_filtering_matrix_factorization.ipynb#singular-value-decomposition).
8. Optionally, make movie recommendations for a new user by providing a list of movie ratings. Just follow the steps provided in 
   + [KNN notebook](./03_collaborative_filtering_similarity.ipynb#predictions-for-a-new-user).
   + [SVD notebook](./03_collaborative_filtering_matrix_factorization.ipynb#predictions-for-a-new-user).

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from surprise import Dataset
from surprise import Reader
from surprise import KNNWithMeans, SVD
from surprise.model_selection import GridSearchCV
from surprise.model_selection import train_test_split
from surprise import accuracy
from collections import defaultdict